In [ ]:
### <font color="red">Import packages </font>

In [52]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### <font color="red">Read the data into csv file</font>

In [53]:
data =pd.read_csv('merged_data.csv')
#data =pd.read_csv('data/bike_sharing_data/merged_data.csv')
#better use a common path, or move the file into git directory, but do not add to commit

In [3]:
data.head()

,Duration,Start date,End date,Start station number,Member type
0,892,2018-02-01 00:01:29,2018-02-01 00:16:21,31312,Member
1,501,2018-02-01 00:04:19,2018-02-01 00:12:40,31089,Member
2,788,2018-02-01 00:05:03,2018-02-01 00:18:12,32048,Member
3,406,2018-02-01 00:18:11,2018-02-01 00:24:57,31224,Member
4,161,2018-02-01 00:18:11,2018-02-01 00:20:53,31126,Member


### <font color="red">Select only the prominant station </font>

In [60]:
data_station_31623=data[data['Start station number']==31623] ##this is the data that we are going to use for EDA

In [61]:
data_station_31623=data_station_31623[["Start date","Member type"]]

In [62]:
data_station_31623.head()

,Start date,Member type
45,2018-01-01 01:13:43,Member
72,2018-01-01 01:49:37,Casual
155,2018-01-01 06:41:10,Member
538,2018-01-01 13:28:19,Member
600,2018-01-01 14:02:33,Member


In [63]:
data_station_31623.tail()

,Start date,Member type
6967823,2020-02-29 22:07:05,Member
6967887,2020-02-29 22:42:54,Member
6967888,2020-02-29 22:43:54,Member
6967902,2020-02-29 22:56:03,Member
6967964,2020-02-29 23:47:02,Member


In [64]:
data_station_31623["Start date"]=pd.to_datetime(data_station_31623["Start date"],format='%Y-%m-%d %H:%M:%S')

In [65]:
data_station_31623['hour of day']=data_station_31623["Start date"].dt.hour

In [66]:
data_station_31623['day of the month']=data_station_31623["Start date"].dt.day

In [69]:
#Note that in Python Monday is 0 unlike Javascript style where Sunday is 0
data_station_31623['day of week']=data_station_31623["Start date"].dt.weekday

#install with pip install holidays
import holidays

holiday_dates_list=[]
#Print all the holidays in US in year 2018, 2019
for ptr in holidays.US(years = 2018).items():
    holiday_dates_list.append(ptr[0])
for ptr in holidays.US(years = 2019).items():
    holiday_dates_list.append(ptr[0])    
us_holidays=tuple(holiday_dates_list)
print(us_holidays)



data_station_31623['is holiday'] = data_station_31623['Start date'].apply(lambda x: 1 if x in us_holidays else 0)

print(data_station_31623['is holiday'].value_counts()) # 121986 1419.

(datetime.date(2018, 1, 1), datetime.date(2018, 1, 15), datetime.date(2018, 2, 19), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4), datetime.date(2018, 9, 3), datetime.date(2018, 10, 8), datetime.date(2018, 11, 11), datetime.date(2018, 11, 12), datetime.date(2018, 11, 22), datetime.date(2018, 12, 25), datetime.date(2019, 1, 1), datetime.date(2019, 1, 21), datetime.date(2019, 2, 18), datetime.date(2019, 5, 27), datetime.date(2019, 7, 4), datetime.date(2019, 9, 2), datetime.date(2019, 10, 14), datetime.date(2019, 11, 11), datetime.date(2019, 11, 28), datetime.date(2019, 12, 25))
0    121986
1      1419
Name: is holiday, dtype: int64


In [70]:
data_station_31623.head()

,Start date,Member type,hour of day,day of the month,date,day of week,is holiday
45,2018-01-01 01:13:43,Member,1,1,2018-01-01,0,1
72,2018-01-01 01:49:37,Casual,1,1,2018-01-01,0,1
155,2018-01-01 06:41:10,Member,6,1,2018-01-01,0,1
538,2018-01-01 13:28:19,Member,13,1,2018-01-01,0,1
600,2018-01-01 14:02:33,Member,14,1,2018-01-01,0,1


In [71]:
data_station_31623['date']=data_station_31623["Start date"].dt.date

In [72]:
count=pd.DataFrame(data_station_31623.groupby(['date','hour of day','Member type', 'day of week', 'is holiday'])\
                   ['hour of day'].count().rename('number of users'))

In [73]:
count=count.reset_index()

In [74]:
casual_hourly=count[count['Member type']=="Casual"][['date','hour of day','number of users','day of week', 'is holiday']]

In [75]:
casual_hourly.head()

,date,hour of day,number of users,day of week,is holiday
0,2018-01-01,1,1,0,1
6,2018-01-01,16,1,0,1
17,2018-01-02,8,1,1,0
36,2018-01-03,7,1,2,0
39,2018-01-03,9,1,2,0


In [76]:
casual_hourly['number of users'].value_counts()

1     2155
2     1066
3      503
4      295
5      156
6       77
7       60
8       52
9       22
10      12
12       9
11       7
13       2
Name: number of users, dtype: int64

In [77]:
casual_hourly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4416 entries, 0 to 17689
Data columns (total 5 columns):
date               4416 non-null object
hour of day        4416 non-null int64
number of users    4416 non-null int64
day of week        4416 non-null int64
is holiday         4416 non-null int64
dtypes: int64(4), object(1)
memory usage: 207.0+ KB


In [78]:
casual_hourly.columns

Index(['date', 'hour of day', 'number of users', 'day of week', 'is holiday'], dtype='object')

In [79]:
member_hourly=count[count['Member type']=="Member"][['date','hour of day','number of users', 'day of week', 'is holiday']]

In [80]:
member_hourly.head()

,date,hour of day,number of users,day of week,is holiday
1,2018-01-01,1,1,0,1
2,2018-01-01,6,1,0,1
3,2018-01-01,13,1,0,1
4,2018-01-01,14,1,0,1
5,2018-01-01,15,3,0,1


In [81]:
member_hourly=member_hourly.reset_index().drop('index',axis=1)

In [82]:
from datetime import timedelta

In [83]:
member_hourly['date_time']=None
for i in range(len(member_hourly)):
    member_hourly['date_time'][i]=timedelta(hours=int(member_hourly['hour of day'].iloc[i]))+pd.to_datetime(member_hourly['date'].iloc[i])
    

In [84]:
member_hourly=member_hourly[['date_time',"number of users"]]

In [85]:
member_hourly.head()

,date_time,number of users
0,2018-01-01 01:00:00,1
1,2018-01-01 06:00:00,1
2,2018-01-01 13:00:00,1
3,2018-01-01 14:00:00,1
4,2018-01-01 15:00:00,3


In [86]:
casual_hourly=casual_hourly.reset_index().drop('index',axis=1)

In [28]:
casual_hourly.index

RangeIndex(start=0, stop=4501, step=1)

In [87]:
casual_hourly['date_time']=None
for i in range(len(casual_hourly)):
    casual_hourly['date_time'][i]=timedelta(hours=int(casual_hourly['hour of day'].iloc[i]))+pd.to_datetime(casual_hourly['date'].iloc[i])

In [88]:
casual_hourly=casual_hourly[['date_time','number of users']]

In [89]:
casual_hourly.head()

,date_time,number of users
0,2018-01-01 01:00:00,1
1,2018-01-01 16:00:00,1
2,2018-01-02 08:00:00,1
3,2018-01-03 07:00:00,1
4,2018-01-03 09:00:00,1


In [90]:
casual_hourly.shape

(4416, 2)

In [91]:
casual_hourly.to_csv('data/bike_sharing_data/casual_hourly.csv',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/bike_sharing_data/casual_hourly.csv'

In [35]:
member_hourly.head()

,date_time,number of users
0,2018-02-01 05:00:00,3
1,2018-02-01 06:00:00,20
2,2018-02-01 07:00:00,23
3,2018-02-01 08:00:00,24
4,2018-02-01 09:00:00,11


In [37]:
member_hourly.to_csv('data/bike_sharing_data/member_hourly.csv',index=False)

In [38]:
member_hourly.shape

(13338, 2)

In [39]:
count_by_date=pd.DataFrame(data_station_31623.groupby(['date','Member type'])['hour of day'].count().rename('number of users'))

In [40]:
count_by_date=count_by_date.reset_index()

In [41]:
count_by_date.head()

,date,Member type,number of users
0,2018-02-01,Casual,5
1,2018-02-01,Member,185
2,2018-02-02,Casual,4
3,2018-02-02,Member,113
4,2018-02-03,Casual,1


In [42]:
casual_daily=count_by_date[count_by_date['Member type']=="Casual"][['date','number of users']]

In [43]:
member_daily=count_by_date[count_by_date['Member type']=="Member"][['date','number of users']]

In [44]:
casual_daily.head()

,date,number of users
0,2018-02-01,5
2,2018-02-02,4
4,2018-02-03,1
7,2018-02-05,3
9,2018-02-06,2


In [45]:
casual_daily.shape

(727, 2)

In [47]:
casual_daily.to_csv('data/bike_sharing_data/casual_daily.csv',index=False)

In [48]:
member_daily.head()

,date,number of users
1,2018-02-01,185
3,2018-02-02,113
5,2018-02-03,35
6,2018-02-04,10
8,2018-02-05,163


In [49]:
member_daily.shape

(759, 2)

In [50]:
member_daily.to_csv('data/bike_sharing_data/member_daily.csv',index=False)